# Model Validation with TFMA

In this lab, we use [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/guide/tfma) to assess the quality of the trained model. This lab covers the following:
1. **Export** evaluation saved model
2. Define **data slices** for analysis
3. Generat **evaluation** the metrics
4. **Visualize** results
5. **Bonus**: Use the **What-If** Tool

In [ ]:
# !pip install -q tensorflow-model-analysis
# !jupyter nbextension enable --py widgetsnbextension --user
# !jupyter nbextension install --py --symlink tensorflow_model_analysis --user
# !jupyter nbextension enable --py tensorflow_model_analysis --user

In [ ]:
# !pip install witwidget
# !jupyter nbextension install --py --symlink --sys-prefix witwidget
# !jupyter nbextension enable --py --sys-prefix witwidget

In [ ]:
import os
import tensorflow as tf
import tensorflow.io as tf_io
import tensorflow_transform as tft
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
from tensorflow_transform.tf_metadata import schema_utils

In [ ]:
WORKSPACE = 'workspace' # you can set to a GCS location
RAW_SCHEMA_LOCATION = os.path.join(WORKSPACE, 'raw_schema.pbtxt')
DATA_DIR = os.path.join(WORKSPACE, 'raw_data')
TRANSFORM_ARTEFACTS_DIR = os.path.join(WORKSPACE, 'transform_artifacts')
DATA_FILES = os.path.join(DATA_DIR,'*.csv')
MODELS_DIR = os.path.join(WORKSPACE, 'models')
MODEL_NAME = 'dnn_classifier'
MODEL_DIR = os.path.join(MODELS_DIR, MODEL_NAME)

### Load TFT Outputs

In [ ]:
transform_output = tft.TFTransformOutput(TRANSFORM_ARTEFACTS_DIR)

## 1. Export Evaluation Saved Model

In [ ]:
HEADER = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
          'marital_status', 'occupation', 'relationship', 'race', 'gender',
          'capital_gain', 'capital_loss', 'hours_per_week',
          'native_country', 'income_bracket']

HEADER_DEFAULTS = [[0], [''], [0], [''], [0], [''], [''], [''], [''], [''],
                   [0], [0], [0], [''], ['']]

TARGET_FEATURE_NAME = 'income_bracket'
TARGET_LABELS = [' <=50K', ' >50K']
WEIGHT_COLUMN_NAME = 'fnlwgt'

### 1.1 Implement eval_input_receiver_fn
This function expect **raw** data interface, then it applies the **transformation**

In [ ]:
def eval_input_receiver_fn():
    
    receiver_tensors = {'examples': tf.placeholder(dtype=tf.string, shape=[None])}
    columns = tf.decode_csv(receiver_tensors['examples'], record_defaults=HEADER_DEFAULTS)
    
    features = dict(zip(HEADER, columns))
    
    for feature_name in features:
        if features[feature_name].dtype == tf.int32:
            features[feature_name] = tf.cast(features[feature_name], tf.int64)
        features[feature_name] = tf.reshape(features[feature_name], (-1, 1))
        
    transformed_features = transform_output.transform_raw_features(features)
    features.update(transformed_features)

    return tfma.export.EvalInputReceiver(
        features=features,
        receiver_tensors=receiver_tensors,
        labels=features[TARGET_FEATURE_NAME]
    )

### 1.2 Export an evaluation saved model
First, we load the estimator...

In [ ]:
import joblib
class Parameters: pass

estimator_file_path = os.path.join(WORKSPACE, 'estimator.joblib')
estimator = joblib.load(estimator_file_path)

In [ ]:
def update_optimizer(initial_learning_rate, decay_steps):
    learning_rate = tf.train.cosine_decay_restarts(
        initial_learning_rate,
        tf.train.get_global_step(),
        first_decay_steps=50,
        t_mul=2.0,
        m_mul=1.0,
        alpha=0.0,
    )
    
    tf.summary.scalar('learning_rate', learning_rate)
    return tf.train.AdamOptimizer(learning_rate=learning_rate)

def metric_fn(labels, predictions):
    
    metrics = {}
    label_index = tf.contrib.lookup.index_table_from_tensor(tf.constant(TARGET_LABELS)).lookup(labels)
    one_hot_labels = tf.one_hot(label_index, len(TARGET_LABELS))
    
    metrics['mirco_accuracy'] = tf.metrics.mean_per_class_accuracy(
        labels=label_index,
        predictions=predictions['class_ids'],
        num_classes=2
    )
    
    return metrics

In [ ]:
TARGET_FEATURE_NAME = 'income_bracket'
TARGET_LABELS = [' <=50K', ' >50K']
WEIGHT_COLUMN_NAME = 'fnlwgt'

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

eval_model_dir = os.path.join(MODEL_DIR, "export/evaluate")
if tf_io.gfile.exists(eval_model_dir):
    tf_io.gfile.rmtree(eval_model_dir)

eval_model_dir = tfma.export.export_eval_savedmodel(
        estimator=estimator,
        export_dir_base=eval_model_dir,
        eval_input_receiver_fn=eval_input_receiver_fn
)

eval_model_dir

In [ ]:
#!saved_model_cli show --dir=${eval_model_dir} --all

## 2. Define Slices for Evaluation

In [ ]:
slice_spec = [
  tfma.slicer.SingleSliceSpec(),
  tfma.slicer.SingleSliceSpec(columns=['occupation'])
]

## 3. Generate evaluation metrics

You can run this on Dataflow by setting the `pipeline_options` parameter.

In [ ]:
eval_result = tfma.run_model_analysis(
    eval_shared_model=tfma.default_eval_shared_model(
        eval_saved_model_path=eval_model_dir,
        example_weight_key=WEIGHT_COLUMN_NAME) , 
    data_location=DATA_FILES, 
    file_format='text', 
    slice_spec=slice_spec,  
    output_path=None
)

In [ ]:
eval_result.slicing_metrics[:5]

## 4. Visalise and analyze evalation results

In [ ]:
tfma.view.render_slicing_metrics(
    result=eval_result, 
    slicing_column='occupation'
)

## 5. Bonus: Using What-If Tool

The [What-if Tool](https://pair-code.github.io/what-if-tool/) makes it easy to efficiently and intuitively explore up to two models' performance on a dataset. Investigate model performances for a range of features in your dataset, optimization strategies and even manipulations to individual datapoint values. All this and more, in a visual way that requires minimal code.

In [ ]:
HEADER = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
               'marital_status', 'occupation', 'relationship', 'race', 'gender',
               'capital_gain', 'capital_loss', 'hours_per_week',
               'native_country', 'income_bracket']

TARGET_FEATURE_NAME = 'income_bracket'
TARGET_LABELS = [' <=50K', ' >50K']
WEIGHT_COLUMN_NAME = 'fnlwgt'
RAW_SCHEMA_LOCATION = os.path.join(WORKSPACE, 'raw_schema.pbtxt')
DATA_FILE = os.path.join(DATA_DIR,'eval.csv')

In [ ]:
import pandas as pd
import numpy as np
SAMPLE = 100

raw_schema = tfdv.load_schema_text(RAW_SCHEMA_LOCATION)
raw_feature_spec = schema_utils.schema_as_feature_spec(raw_schema).feature_spec
data_frame = pd.read_csv(DATA_FILE, names=HEADER).sample(n=SAMPLE)

def df_to_examples(df, columns):
    examples = []
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if df[col].dtype is np.dtype(np.int64):
                example.features.feature[col].int64_list.value.append(int(row[col]))
            elif df[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            elif row[col] == row[col]:
                example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
        examples.append(example)
    return examples

examples = df_to_examples(data_frame, HEADER)

In [ ]:
from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

config_builder = WitConfigBuilder(examples) \
    .set_estimator_and_feature_spec(estimator, raw_feature_spec) \
    .set_target_feature(TARGET_FEATURE_NAME) \
    .set_label_vocab(TARGET_LABELS)

WitWidget(config_builder, height=800)